In [1]:
from skimage.transform import (hough_line, hough_line_peaks)
import numpy as np
import cv2
from statistics import mean

In [2]:
croppedImage = r'./cropped_images/DSC_0637_cropped.jpg'

img_gray = cv2.imread(croppedImage) #image is already grayscale, read into python with open cv
img_blur = cv2.GaussianBlur(img_gray, (3,3), 0) #perform a gaussian blur to help for edge detection, this smooths the edges

In [11]:
# Sobel Edge Detection
# sobel edge detection uses a 2D derivative mask to detect the edges in an image, both vertical and horizontal masks are used
# for the purposes of better identitfying the angles, ddepth is the desired depth of the destination image.

sobelxy = cv2.Sobel(src=img_blur, ddepth=cv2.CV_64F, dx=1, dy=1, ksize=5) # Combined X and Y Sobel Edge Detection
# Display Sobel Edge Detection Images
cv2.imshow('Sobel X Y using Sobel() function', sobelxy)
cv2.waitKey(0)

-1

In [4]:
from skimage.transform import (hough_line, hough_line_peaks)
import numpy as np
import cv2

# Compute arithmetic mean for the hough transformation
image = np.mean(sobelxy, axis=2)

# Perform Hough Transformation to detect lines
hspace, angles, distances = hough_line(image) #hough transform can be used to detect straight lines and their angles
#transforms into polar coordinates with a rho and a theta

# Find angle using a function that identifies peaks in the hough transform, finds prominent lines and saves the angles
angle=[]
for _, a , dist in zip(*hough_line_peaks(hspace, angles, distances)):
    angle.append(a)

# Obtain angle in degrees for each line
angles = [a*180/np.pi for a in angle]

angles

[33.687150837988824,
 -33.687150837988824,
 -26.648044692737443,
 26.64804469273743,
 23.631284916201107,
 -23.63128491620112,
 18.60335195530725,
 -19.60893854748604,
 14.581005586592168,
 -13.575418994413418,
 -16.592178770949722,
 -8.547486033519558,
 -10.5586592178771,
 -3.5195530726257,
 6.536312849162005,
 4.525139664804464,
 -44.7486033519553,
 -0.5027932960893948,
 -45.75418994413408,
 -54.80446927374302,
 -57.821229050279335,
 -60.83798882681564,
 66.87150837988825,
 -67.87709497206704,
 -72.9050279329609,
 72.90502793296088,
 75.92178770949721,
 76.92737430167597,
 -78.93854748603351,
 -76.92737430167597,
 -84.97206703910614,
 90.0,
 86.98324022346368,
 -86.98324022346368,
 -90.0,
 -88.99441340782121]

In [5]:
from statistics import mean
#change negative angle outputs to their positive counterpart, angles measured from counterclockwise between -90 and 90
angles = [a if (a > 0) else (180 + a) for a in angles]


# from prior measurements, it is known that the maximum braid angle is less than 65degrees and the minimum braid 
# angle is more than 15 degrees, this allows for the creation of the bounds that no angle should be within 15 degrees of 90,
# and no angle should be within 25 degrees of 0 or 180

# we follow this for the further filtration of the angle outputs, knowing all the images are in the same orientation

angles = [a for a in angles if (a > (90+15) or a < (90-15))] #filters out angles close to 90 for min braid angle
angles = [a for a in angles if (a < (180-25) and a > (0+25))] #filters out angles close to 0 or 180 for max braid angle

angRight = [a for a in angles if a < 90] #collects all the angles on the "right side"
angLeft = [a for a in angles if a > 90] #collects all the angles on the "left side"

# creates a list of all the angles between all the lines on the right side and all the lines on the left side to find a list of all
# possible angles, the braid angle is half the value
braidAngle = []
for val in angRight:
    for val2 in angLeft:
        ang = (val2-val)/2
        braidAngle.append(ang)

# creates a list of all the angles of the lines as compared to the vertical since this is another way to find the braid angle
# this method does not as neatly account for slight variations in the angle the picture was taken
braidAngleVert = []
for val in angRight:
    ang = 90 - val
    braidAngleVert.append(ang)
for val in angLeft:
    ang = val - 90
    braidAngleVert.append(ang)

braidAngle = np.array(braidAngle)
braidAngleVert = np.array(braidAngleVert)
# finds the mean of both of the lists and then adds it to the overall list of angles for each image
braidAngleAvg = np.mean(braidAngle)
braidAngleVertAvg = np.mean(braidAngleVert)

In [6]:
angles

[33.687150837988824,
 146.31284916201116,
 153.35195530726256,
 26.64804469273743,
 135.2513966480447,
 134.24581005586592,
 125.19553072625698,
 122.17877094972067,
 119.16201117318437,
 66.87150837988825,
 112.12290502793296,
 107.0949720670391,
 72.90502793296088]

In [8]:
braidAngleVertAvg

38.831113021057156

In [9]:
braidAngleAvg

39.14804469273744